In [1]:
# -*- coding: utf-8 -*-
# Kaggle Notebook cell: faithful extraction of the first two columns from HTML tables in a Markdown file

from pathlib import Path
from typing import List
import pandas as pd
from bs4 import BeautifulSoup

# ===== 1) 配置 =====
# 例：/kaggle/input/your-dataset/NSFC-2025-management.md 或 /kaggle/working/xxx.md
INPUT_MD = "/kaggle/input/nsfc2025-management/NSFC-2025-management-raw-7tables.md"   # <-- 改为你的 md 路径
OUTPUT_CSV = "/kaggle/working/extraction_from_md.csv"

# ===== 2) 工具函数（仅提取 <table>，不做任何内容处理）=====
def html_tables_from_markdown(md_text: str) -> List[str]:
    """直接在全文里查找 <table> 节点。"""
    soup = BeautifulSoup(md_text, "html.parser")
    tables = soup.find_all("table")
    return [str(t) for t in tables]

def parse_html_table_to_df_first2cols(html_table_str: str) -> pd.DataFrame:
    """
    用 pandas.read_html(flavor='bs4') 解析单个 HTML 表格，**仅截取前两列**。
    不做任何其它处理：不 ffill，不填空，不去空白，不删表头。
    """
    try:
        dfs = pd.read_html(html_table_str, flavor="bs4")
    except Exception:
        dfs = []
    if not dfs:
        return pd.DataFrame()

    df = dfs[0]

    # 若列少于2列，按“忠实”原则：原本就只有一列/零列，则补出空列以便成两列（保持结构一致）。
    # 注意：这是结构层面的补齐，不改变原有单元格内容；补出的列为空值（NaN）。
    if df.shape[1] == 0:
        df["__c0__"] = pd.NA
        df["__c1__"] = pd.NA
    elif df.shape[1] == 1:
        df["__c1__"] = pd.NA

    # 仅保留前两列；**不改变这些列中的任何值**（NaN 保持 NaN，空串保持空串，空白保持空白）
    return df.iloc[:, :2]

def extract_first_two_cols_faithful(md_path: str) -> pd.DataFrame:
    p = Path(md_path)
    if not p.exists():
        raise FileNotFoundError(f"找不到文件：{p}")

    text = p.read_text(encoding="utf-8", errors="ignore")
    table_html_list = html_tables_from_markdown(text)

    parts = []
    for t in table_html_list:
        df2 = parse_html_table_to_df_first2cols(t)
        if not df2.empty or df2.shape[0] > 0:
            parts.append(df2)

    if not parts:
        # 仍然返回两列的空 DataFrame，以便后续保存 CSV 结构固定
        return pd.DataFrame(columns=["col1", "col2"])

    out = pd.concat(parts, ignore_index=True)

    # 为了便于识别，两列统一命名；**仅改列名，不改数据**
    out.columns = ["col1", "col2"]
    return out

# ===== 3) 运行与保存 =====
df_out = extract_first_two_cols_faithful(INPUT_MD)

# 预览前若干行（注意：这里仅展示，不做任何数据修改）
display(df_out.head(12))

# 保存：不改变数据；pandas 会将 NaN 写成空字段（CSV 标准做法）
Path(OUTPUT_CSV).parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print(f"✅ 完成：共 {len(df_out)} 行（含可能两列皆空的行）")
print(f"📄 CSV 已保存：{OUTPUT_CSV}")


/tmp/ipykernel_37/807212102.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html_table_str, flavor="bs4")
/tmp/ipykernel_37/807212102.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html_table_str, flavor="bs4")
/tmp/ipykernel_37/807212102.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html_table_str, flavor="bs4")
/tmp/ipykernel_37/807212102.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html_table

col1                          col2
0   负责人                          项目名称
1   薛润坡          基于内生经济政策的经济发展理论与定量研究
2   张庆昭              高维异质图模型的若干重要问题研究
3   钟齐先                 基于深度学习的生存数据分析
4   王中雷               复杂样本的稳健统计推断理论探索
5   方匡南         多源高维正例无标签数据的联邦学习方法及应用
6    冷旋        尾部强化的分位数回归方法及在风险管理中的应用
7   李迎星  数智驱动的资产收益率预测与投资组合优化：理论、算法与应用
8    耿森    手机过度使用干预机制设计与福利效应评估：社会比较视角
9   李梦玲  公共资源分配中的优先权制度与匹配机制：激励优化与市场设计
10   郭晔       供应链关联、银行信贷与货币政策实体经济效应研究
11  赵向琴   气候灾难冲击、生产-金融网络溢出和气候韧性提升路径研究

✅ 完成：共 963 行（含可能两列皆空的行）
📄 CSV 已保存：/kaggle/working/extraction_from_md.csv
